In [91]:
import pandas as pd
import regex as re
import numpy as np
import os

# Read in a plain text file
with open(os.path.join("trial1.txt"), "r") as f:
    text1 = f.read()
with open(os.path.join("trial2.txt"), "r") as f:
    text2= f.read()
with open(os.path.join("trial3.txt"), "r") as f:
    text3= f.read()
    
    
with open(os.path.join("web scraping04-1350.pdf.txt"), "r") as f:
    text4= f.read()
with open(os.path.join("web scraping05-1056.pdf.txt"), "r") as f:
    text5= f.read()
with open(os.path.join("web scraping05-1074.pdf.txt"), "r") as f:
    text6= f.read()
with open(os.path.join("web scraping05-1120.pdf.txt"), "r") as f:
    text7= f.read()
with open(os.path.join("web scraping07-1125.pdf.txt"), "r") as f:
    text8= f.read()
with open(os.path.join("web scraping07-1437.pdf.txt"), "r") as f:
    text9= f.read()
with open(os.path.join("web scraping08-322.pdf.txt"), "r") as f:
    text10= f.read()
with open(os.path.join("web scraping10-1121.pdf.txt"), "r") as f:
    text11= f.read()
with open(os.path.join("web scraping11-262.pdf.txt"), "r") as f:
    text12= f.read()
with open(os.path.join("web scraping12-1408.pdf.txt"), "r") as f:
    text13= f.read()
with open(os.path.join("web scraping14-1516.pdf.txt"), "r") as f:
    text14= f.read()
with open(os.path.join("web scraping17-5716.pdf.txt"), "r") as f:
    text15= f.read()

In [92]:
t = [text1,text2,text3]
#print(text1)
t1 = [text4,text5,text6,text7,text8,text9,text10,text11,text12,text13,text14,text15]

In [93]:
cleaned = []
for txt in t1:
    clean = re.sub('\xad','',txt)
    clean = re.sub('\n','',clean)
    clean = re.sub('\\\\','',clean)
    cleaned += [clean]

### **Finding Justices Present**

In [94]:
def unique(lst):
    uni = []
    for i in lst:
        if i not in uni:
            uni += [i]
    return uni

In [95]:
def case_no(txt):
    return unique(re.findall('No.\s\d+[-]*\d+',txt))

In [96]:
for i in cleaned:
    print(case_no(i))

['No. 04-1350']
['No. 05-1056']
['No. 05-1074']
['No. 05-1120']
['No. 07-1125']
['No. 07-1437']
['No. 08-322']
['No. 10-1121']
['No. 11-262']
['No. 121408']
['No. 141516']
['No. 17-5716']


In [97]:
def justices(texts):
    d = {}
    for txt in texts:
        num = case_no(txt)[0]
        j = re.findall('JUSTICE[A-Z\s]+:',txt)
        justice = sorted(unique(j))
        cleaned_list = [justice[i][:-1] for i in range(len(justice))]
        d[num] = cleaned_list
    return d

## **Date, Year**

In [8]:
def date(text):
    return re.findall('\w+\s+\d+,\s+\d{4}',text)[0]

In [137]:
def year(text):
    return re.findall('\d\d\d\d',date(text))

In [11]:
#This doesn't work
def title(text):
    s = re.findall('[-\s]*[\w\n\s\d.,\/#!$%\^&\*;:{}=\-_`~()]*[-\s]*',text)[0]
    s1 = re.findall('\w+\s\w+\s\w+',s)
    first = re.findall('\s\w+',s1[0])
    first = ''.join(first)
    last = re.findall('\s\w+',s1[1])
    last = ''.join(last)
    title = first + ' v.'+last
    return title

## **Appearances**

In [101]:
def appearances(text):
    app = re.findall('APPEARANCES:[\s\S]*?Reporting',text)[0]
    app = re.findall('[\s\S]*?;[\s\S]*?\.',app)
    app = [re.sub('\d','',app[i]) for i in range(len(app))]
    remove_appearance = re.sub('APPEARANCES:\s','',app[0])
    app[0] = remove_appearance
    app = [re.sub('[\s\s]+',' ',app[i]) for i in range(len(app))]
    return app

In [102]:
for i in cleaned:
    app = appearances(i)
    for j in app:
        print(j)
    print('-----------------------')

JAMES W. DABNEY, ESQ., New York, N.Y.; on behalf of Petitioner.
 THOMAS G. HUNGAR, ESQ., Deputy Solicitor General, Department of Justice, Washington, D.C.; on behalf of the United States, as amicus curiae, supporting Petitioner.
-----------------------
THEODORE B. OLSON, ESQ., Washington, D.C.; on behalf of Petitioner.
 DARYL JOSEFFER, ESQ., Assistant to the Solicitor General, Department of Justice, Washington, D.C.; On behalf of the United States, as amicus curiae, supporting Petitioner.
 SETH P. WAXMAN, ESQ., Washington, D.C.; on behalf of Respondent.
-----------------------
KEVIN K. RUSSELL, ESQ., Washington, D.C.; on behalf of the Petitioner.
 GLEN D. NAGER, ESQ., Washington, D.C.; on behalf of the Respondent.
 IRVING L. GORNSTEIN, ESQ., Assistant to the Solicitor General, Department of Justice, Washington, D.C.; as amicus curiae on behalf of the Respondent.
-----------------------
JAMES R. MILKEY, ESQ., Assistant Attorney General, Boston, Mass; on behalf of Petitioners.
 GREGORY C

## **Sentiment Analysis**

In [14]:
!pip install vaderSentiment

  Using cached https://files.pythonhosted.org/packages/86/9e/c53e1fc61aac5ee490a6ac5e21b1ac04e55a7c2aba647bb8411c9aadf24e/vaderSentiment-3.2.1-py2.py3-none-any.whl


In [15]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [16]:
#DEMO
sentences = ["VADER is smart, handsome, and funny.",  # positive sentence example
             "VADER is smart, handsome, and funny!",  # punctuation emphasis handled correctly (sentiment intensity adjusted)
             "VADER is very smart, handsome, and funny.", # booster words handled correctly (sentiment intensity adjusted)
             "VADER is VERY SMART, handsome, and FUNNY.",  # emphasis for ALLCAPS handled
             "VADER is VERY SMART, handsome, and FUNNY!!!", # combination of signals - VADER appropriately adjusts intensity
             "VADER is VERY SMART, uber handsome, and FRIGGIN FUNNY!!!", # booster words & punctuation make this close to ceiling for score
             "VADER is not smart, handsome, nor funny.",  # negation sentence example
             "The book was good.",  # positive sentence
             "At least it isn't a horrible book.",  # negated negative sentence with contraction
             "The book was only kind of good.", # qualified positive sentence is handled correctly (intensity adjusted)
             "The plot was good, but the characters are uncompelling and the dialog is not great.", # mixed negation sentence
             "Today SUX!",  # negative slang with capitalization emphasis
             "Today only kinda sux! But I'll get by, lol", # mixed sentiment example with slang and constrastive conjunction "but"
             "Make sure you :) or :D today!",  # emoticons handled
             "Catch utf-8 emoji such as such as 💘 and 💋 and 😁",  # emojis handled
             "Not bad at all"  # Capitalized negation
             ]
analyzer = SentimentIntensityAnalyzer()
for sentence in sentences:
    vs = analyzer.polarity_scores(sentence)
    print("{:-<65} {}".format(sentence, str(vs)))

VADER is smart, handsome, and funny.----------------------------- {'neg': 0.0, 'neu': 0.254, 'pos': 0.746, 'compound': 0.8316}
VADER is smart, handsome, and funny!----------------------------- {'neg': 0.0, 'neu': 0.248, 'pos': 0.752, 'compound': 0.8439}
VADER is very smart, handsome, and funny.------------------------ {'neg': 0.0, 'neu': 0.299, 'pos': 0.701, 'compound': 0.8545}
VADER is VERY SMART, handsome, and FUNNY.------------------------ {'neg': 0.0, 'neu': 0.246, 'pos': 0.754, 'compound': 0.9227}
VADER is VERY SMART, handsome, and FUNNY!!!---------------------- {'neg': 0.0, 'neu': 0.233, 'pos': 0.767, 'compound': 0.9342}
VADER is VERY SMART, uber handsome, and FRIGGIN FUNNY!!!--------- {'neg': 0.0, 'neu': 0.294, 'pos': 0.706, 'compound': 0.9469}
VADER is not smart, handsome, nor funny.------------------------- {'neg': 0.646, 'neu': 0.354, 'pos': 0.0, 'compound': -0.7424}
The book was good.----------------------------------------------- {'neg': 0.0, 'neu': 0.508, 'pos': 0.492, 'co

In [17]:
sent1 = re.sub('\d','',text1.replace('\n',' '))
sents = re.findall('[A-Z]+:[\w\n\s\d\.,\/#!\$%\'\^&\\\"*;:{}=\-\_`~()]*',sent1)
#print(text1)
sents[0]

'APPEARANCES:     THOMAS H. DUPREE, JR., ESQ., Washington, D.C.; on     behalf of the Petitioners.     RACHEL P. KOVNER, Assistant to the Solicitor General,     Department of Justice, Washington, D.C.; on     behalf of the Respondent.        Heritage Reporting Corporation   \x0cOfficial     C O N T E N T S    ORAL ARGUMENT OF:    THOMAS H. DUPREE, JR., ESQ.    On behalf of the Petitioners    ORAL ARGUMENT OF:    RACHEL P. KOVNER, ESQ.    On behalf of the Respondent    REBUTTAL ARGUMENT OF:    THOMAS H. DUPREE, ESQ.    On behalf of the Petitioners                  Heritage Reporting Corporation   PAGE:        \x0cOfficial     P R O C E E D I N G S     (: a.m.)     CHIEF JUSTICE ROBERTS:  We\'ll hear     argument first this morning in Case -,     Wisconsin Central versus the United States.     Mr. Dupree.     ORAL ARGUMENT OF THOMAS H. DUPREE, JR.     ON BEHALF OF THE PETITIONERS      MR. DUPREE:  Mr. Chief Justice, and   may it please the Court:     The Railroad Retirement Tax Act levie

# **Building the DataFrame**

In [154]:
##Running functions over files

cases = [case_no(i)[0] for i in cleaned]
justice = [justices(cleaned).get(num) for num in cases]
dates = [date(i) for i in cleaned]
years = [year(i)[0] for i in cleaned]
people = [appearances(i) for i in cleaned]

In [150]:
data = pd.DataFrame({'Case No': cases,'Justices':justice,'Date': dates,'Year':years,'Appearances':people})

In [158]:
data.head()

,Case No,Justices,Date,Year,Appearances
0,No. 04-1350,"[JUSTICE ALITO, JUSTICE BREYER, JUSTICE GINSBU...","November 28, 2006",2006,"[JAMES W. DABNEY, ESQ., New York, N.Y.; on beh..."
1,No. 05-1056,"[JUSTICE ALITO, JUSTICE BREYER, JUSTICE GINSBU...","February 21, 2007",2007,"[THEODORE B. OLSON, ESQ., Washington, D.C.; on..."
2,No. 05-1074,"[JUSTICE ALITO, JUSTICE BREYER, JUSTICE GINSBU...","November 27, 2006",2006,"[KEVIN K. RUSSELL, ESQ., Washington, D.C.; on ..."
3,No. 05-1120,"[JUSTICE ALITO, JUSTICE BREYER, JUSTICE GINSBU...","November 29, 2006",2006,"[JAMES R. MILKEY, ESQ., Assistant Attorney Gen..."
4,No. 07-1125,"[JUSTICE BREYER, JUSTICE GINSBURG, JUSTICE KEN...","December 2, 2008",2008,"[CHARLES A. ROTHFELD, ESQ., Washington, D.C.; ..."


In [157]:
pd.to_datetime(data['Date'])

0    2006-11-28
1    2007-02-21
2    2006-11-27
3    2006-11-29
4    2008-12-02
5    2009-02-24
6    2009-04-29
7    2012-01-10
8    2012-03-21
9    2014-01-14
10   2016-01-12
11   2018-03-27
Name: Date, dtype: datetime64[ns]